https://www.fahdmirza.com/2024/08/free-llm-dataset-creation-with-ollama.html

## Prepare sentences to paraphrase

In [1]:
import json
import ollama
import os

In [2]:
path = "..//data//external//cadec//text"
dir_list = os.listdir(path)

In [3]:
def long_sentence(s):
    return len(s.split()) >= 5

In [18]:
file_path = 'example.txt'
files_text= []

for f in dir_list[:10]:
    with open(f'{path}//{f}', 'r') as file:
        for line in file.readlines():
                files_text.append(line)

sentences = list(filter(long_sentence, files_text))

for s in sentences:
    print(s)

I feel a bit drowsy & have a little blurred vision, so far no gastric problems.

I've been on Arthrotec 50 for over 10 years on and off, only taking it when I needed it.

Due to my arthritis getting progressively worse, to the point where I am in tears with the agony, gp's started me on 75 twice a day and I have to take it.

every day for the next month to see how I get on, here goes.

So far its been very good, pains almost gone, but I feel a bit weird, didn't have that when on 50.

Brilliant, I have a new lease of life, i walk up & down steps properly, no longer sideways like a toddler, hip pain as gone other than if i jar it.

no side effects for the first two months .

then vaginal bleeding 2 wks after menstral cycle.

canker sores in my mouth.

been off for 1 week still have bleeding.

helped my pain alot .

too scared to take this drug again.

1st pill taken with food, a few hours after i experienced shortness of breath, a sense of depression, cramping, upset stomache will stop t

## Write prompt

In [5]:
def make_llama_3_prompt(user, system="", assistant=""):
    system_prompt = ""
    if system:
        system_prompt = (
            f"<|start_header_id|>system<|end_header_id|>\n\n{system}<|eot_id|>"
        )
    
    user_prompt = f"<|start_header_id|>user<|end_header_id|>\n\n{user}<|eot_id|>"
    assistant_prompt = f"<|start_header_id|>assistant<|end_header_id|>\n\n{assistant}<|eot_id|>" if assistant else "<|start_header_id|>assistant<|end_header_id|>\n\n"
    
    return f"<|begin_of_text|>{system_prompt}{user_prompt}{assistant_prompt}"

In [20]:
def generate_paraphrase(sentence):
    system = "You are an english text editor with 10 years of experience in paraphrasing sentences. \n"
    system += "Consider the following sentences and their paraphrases:\n"
    
    sentence_1 = "I was prescribed a medication for bone and joint pain associated with Lupus."
    paraphrase_1 = "I was prescribed a drug to treat the joint and bone ache caused by lupus."
    
    sentence_2 = "I have had no recognizable side effects."
    paraphrase_2 = "I have not experienced any noticeable side effects."

    sentence_3 = "The drug almost completely wiped out the pain from the herniated discs, and has replaced patients' usual narcotic pain relief for back."
    paraphrase_3 = "The drug has almost completely eliminated the pain caused by the herniated discs, providing an alternative to the usual narcotic pain relief for back pain."
    
    system += "Sentence: " + sentence_1 + "\n"
    system += "Paraphrase: " + paraphrase_1 + "\n"
    system += "Sentence: " + sentence_2 + "\n"
    system += "Paraphrase: " + paraphrase_2 + "\n"
    system += "Sentence: " + sentence_3 + "\n"
    system += "Paraphrase: " + paraphrase_3 + "\n"
    
    user = "Based on the above examples, write a paraphrase of the following sentence: " + sentence + "\n"
    user += "Format the sentence and paraphrase as a JSON object, i.e.\n"
    user += '{"sentence" : str, "paraphrase": str }.\n'
   
    user += """\
            Make sure to only return a well-written paraphrase of the given sentence. \
            Don't give any comments. Just return a sentence and a paraphrase as JSON objects. \
            Make sure paraphrase has the same meaning as the sentence. \
            Make sure all medical entities from original sentence are also present in the paraphrase (in original or paraphrased form e.g 'ache' instead of 'pain'). \
            Make sure there are no medical entities in paraphrase, that are not present in the original sentence.
            """
    system += sentence
    
    prompt = make_llama_3_prompt(user, system)

    # Generate the result from the model
    result = ollama.generate(model='llama3.2', prompt=prompt)

    # Inspect and parse the result['response']
    response_str = result['response']
    try:
        response_dict = json.loads(response_str)
    except json.JSONDecodeError as e:
        print("Failed to parse response as JSON:", e)
        response_dict = {}

    return response_dict

## Run prompt

In [ ]:
all_paraphrases = []

for s in sentences:
    
    paraphrase = generate_paraphrase(s)
    all_paraphrases.append(paraphrase)

In [22]:
all_paraphrases

[{'sentence': 'I feel a bit drowsy & have a little blurred vision, so far no gastric problems.',
  'paraphrase': 'So far, I am experiencing some dizziness and slight double vision, with no signs of stomach issues.'},
 {'sentence': "I've been on Arthrotec 50 for over 10 years on and off, only taking it when I needed it.",
  'paraphrase': 'I have relied on Arthrotec 50 sporadically for over a decade.'},
 {'sentence': "Due to my arthritis getting progressively worse, to the point where I am in tears with the agony, gp's started me on 75 twice a day and I have to take it.",
  'paraphrase': 'Because of worsening arthritis, which is causing me extreme distress, doctors prescribed me 75 mg twice a day, which I must now ingest regularly.'},
 {'sentence': 'every day for the next month to see how I get on, here goes.',
  'paraphrase': 'I will commit to checking my progress every day for the next month and proceed as planned.'},
 {'sentence': "So far its been very good, pains almost gone, but I f

## Save output to json lines

In [7]:
def save_to_jsonl(data, file_path):
    with open(file_path, 'a') as f:
        for entry in data:
            f.write(json.dumps(entry) + '\n')

In [24]:
output_file_path = '../data/processed/paraphrases.jsonl'

save_to_jsonl(all_paraphrases, output_file_path)
print(f"Saved {len(all_paraphrases)} questions and queries to {output_file_path}")

Saved 40 questions and queries to ../data/processed/paraphrases.jsonl
